In [1]:
import telegram
from telegram import ReplyKeyboardMarkup
from telegram.ext import Updater
from telegram.ext import CommandHandler, MessageHandler, Filters, ConversationHandler
from telegram.ext import BaseFilter

from db import Event, User, connect_default

token = ''
bot = telegram.Bot(token)
updater = Updater(token=token)
dispatcher = updater.dispatcher

meta, session, base, db = connect_default()
from sqlalchemy import exists

import logging

logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

# input data
organizations = ["1", "2"]
profiles = ["Бизнес", "Карьера", "Технологии", "Искуство", "Chill"]
dict_class_names = {"1": "chose_1", "2": 'chose_2', "Бизнес": 'th_business',
                    "Карьера": 'th_career', "Технологии": 'th_tech', "Искуство": 'th_arts', "Chill": 'th_chill'}

choice = {}


def start(bot, update):
    uid = update.message.chat_id

    if not session.query(exists().where(User.uid == uid)).scalar():
        text = "/help"
        keyboard = [["Подписаться", "Помощь"]]
        reply_markup = ReplyKeyboardMarkup(
            keyboard, one_time_keyboard=True, resize_keyboard=True)
        bot.send_message(chat_id=update.message.chat_id, text=text, parse_mode='Markdown',
                         reply_markup=reply_markup)
        session.add(User(uid=uid))
        session.commit()
    else:
        text = "Ты уже подписан, а я работаю над анонсами для тебя:)"
        bot.send_message(chat_id=uid, text=text)
        help_function(bot, update)


def subscribe(bot, update):
    uid = update.message.chat_id

    text = "Ты можешь настроить подписку сперва по вузам и клёвым организация, а затем по тематике. Выбери вузы и " \
           "организации, откуда хочешь получать уведомления о событиях. "
    reply_markup = ReplyKeyboardMarkup(
        keyboard=[organizations], one_time_keyboard=True, resize_keyboard=True)
    bot.send_message(chat_id=uid, text=text, reply_markup=reply_markup)

    choice[update.message.chat_id] = [[], []]
    user = session.query(User).filter_by(uid=uid).first()
    for key, value in dict_class_names.items():
        setattr(user, value, False)
    setattr(user, 'is_sub', False)
    session.commit()


def choose_org(bot, update):
    choose = choice[update.message.chat_id][0]
    uid = update.message.chat_id

    if update.message.text in organizations:
        choose.append(update.message.text)
        keyboard = []
        reply_markup = ReplyKeyboardMarkup(
            keyboard=[keyboard], one_time_keyboard=True, resize_keyboard=True)
        for org in organizations:
            if org not in choose:
                keyboard.append(org)
        if len(keyboard) == 0:
            text = "Ты выбрал все организации, на которые доступна подписка сегодня:)"
            bot.send_message(chat_id=uid, text=text)
            return choose_profile(bot, update)
        elif len(keyboard) != 0 and len(keyboard) < len(organizations):
            text = "Что добавить?"
            keyboard.append("Закончить")
            bot.send_message(chat_id=uid, text=text,
                             reply_markup=reply_markup, resize_keyboard=True)
    elif update.message.text not in organizations:
        keyboard = []
        reply_markup = ReplyKeyboardMarkup(
            keyboard=[keyboard], one_time_keyboard=True, resize_keyboard=True)
        for org in organizations:
            if org not in choose:
                keyboard.append(org)
        if len(keyboard) == 0:
            text = "Ты выбрал все организации, на которые доступна подписка сегодня:)"
            bot.send_message(chat_id=update.message.chat_id, text=text)
            return choose_profile(bot, update)
        elif len(keyboard) != 0 and len(keyboard) < len(organizations):
            text = "Что добавить?"
            keyboard.append("Закончить")
            bot.send_message(chat_id=update.message.chat_id, text=text,
                             reply_markup=reply_markup, resize_keyboard=True)


def choose_profile(bot, update):
    uid = update.message.chat_id
    choose = choice[uid][1]
    if update.message.text in profiles:
        choose.append(update.message.text)
        text = "Что-нибудь ещё?"
        keyboard = []
        reply_markup = ReplyKeyboardMarkup(
            keyboard=[keyboard], one_time_keyboard=True, resize_keyboard=True)
        for profile in profiles:
            if profile not in choose:
                keyboard.append(profile)
        if len(keyboard) < len(profiles):
            keyboard.append("Завершить")
            bot.send_message(chat_id=uid,
                             text=text, reply_markup=reply_markup)
    elif update.message.text == "Завершить":
        user = session.query(User).filter_by(uid=uid).first()
        for key, value in dict_class_names.items():
            if (key in choice[uid][0]) or (key in choice[uid][1]):
                setattr(user, value, True)
        setattr(user, 'is_sub', True)
        session.commit()

        text = "Отлично! Теперь ты будешь получать по будням анонсы событий на «сегодня» и «завтра», " \
               "а по воскресеньям – на всю неделю. Ты подписан на: {}. " \
               "Жми /settings, чтобы всё удалить и настроить заново).".format(list(choice.values()))
        bot.send_message(chat_id=update.message.chat_id, text=text)
    else:
        text = "А какие события тебя больше всего интересуют?"
        keyboard = []
        reply_markup = ReplyKeyboardMarkup(
            keyboard=[keyboard], one_time_keyboard=True, resize_keyboard=True)
        for profile in profiles:
            if profile not in choose:
                keyboard.append(profile)
        if len(keyboard) == len(profiles):
            bot.send_message(chat_id=update.message.chat_id,
                             text=text, reply_markup=reply_markup)
        else:
            keyboard.append("Завершить")
            bot.send_message(chat_id=update.message.chat_id,
                             text=text, reply_markup=reply_markup)


def settings(bot, update):
    text = "Подпишись, и мы составим твои настройки"
    keyboard = [["Подписаться"]]
    reply_markup = ReplyKeyboardMarkup(
        keyboard, one_time_keyboard=True, resize_keyboard=True)
    bot.send_message(chat_id=update.message.chat_id,
                     text=text, reply_markup=reply_markup)


def unknown(bot, update):
    text = "Такой команды нет. Выбери комманду из списка:)"
    bot.send_message(chat_id=update.message.chat_id, text=text)
    help_function(bot, update)


def error(bot, update, TelegramError):
    print(TelegramError)


def help_function(bot, update):
    text = """
        Вот список команд, который поможет тебе пользоваться ботом:

    /start - Начать
    /subscribe - Подписаться
    /settings - Настройки
        """
    bot.send_message(chat_id=update.message.chat_id, text=text)




class SubscribeFilter(BaseFilter):
    def filter(self, message):
        return 'Подписаться' in message.text


class OrganizationFilter(BaseFilter):
    def filter(self, message):
        if message.text in organizations:
            for i in organizations:
                if i not in choice[message.chat_id][0]:
                    if i in message.text:
                        return True
        elif "Добавить" in message.text:
            return True
        else:
            return False


class ProfileFilter(BaseFilter):
    def filter(self, message):
        if message.text in profiles:
            for i in profiles:
                if i not in choice[message.chat_id][1]:
                    if i in message.text:
                        return True
        if "Закончить" in message.text or "Завершить" in message.text:
            return True
        else:
            return False


filter_subscribe = SubscribeFilter()
filter_organization = OrganizationFilter()
filter_profile = ProfileFilter()
start_handler = CommandHandler('start', start)
subscribe_handler = CommandHandler('subscribe', subscribe)
settings_handler = CommandHandler('settings', settings)
subscribe_handler2 = MessageHandler(filter_subscribe, subscribe)
chooseOrg_handler = MessageHandler(filter_organization, choose_org)
chooseProfile_handler = MessageHandler(filter_profile, choose_profile)
help_function_handler = CommandHandler('help', help_function)

unknown_handler = MessageHandler(Filters.command, unknown)
error_handler = MessageHandler(Filters.all, error)
dispatcher.add_handler(start_handler)
dispatcher.add_handler(subscribe_handler)
dispatcher.add_handler(settings_handler)
dispatcher.add_handler(subscribe_handler2)
dispatcher.add_handler(chooseOrg_handler)
dispatcher.add_handler(chooseProfile_handler)
dispatcher.add_handler(help_function_handler)
dispatcher.add_handler(unknown_handler)
dispatcher.add_error_handler(error_handler)
updater.start_polling()
# updater.idle()


/home/eamag/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


2018-04-20 22:11:19,435 INFO sqlalchemy.engine.base.Engine select version()
2018-04-20 22:11:19,435 INFO sqlalchemy.engine.base.Engine {}
2018-04-20 22:11:19,456 INFO sqlalchemy.engine.base.Engine select current_schema()
2018-04-20 22:11:19,457 INFO sqlalchemy.engine.base.Engine {}
2018-04-20 22:11:19,461 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-04-20 22:11:19,462 INFO sqlalchemy.engine.base.Engine {}
2018-04-20 22:11:19,501 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-04-20 22:11:19,501 INFO sqlalchemy.engine.base.Engine {}
2018-04-20 22:11:19,502 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2018-04-20 22:11:19,503 INFO sqlalchemy.engine.base.Engine {}
2018-04-20 22:11:19,505 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

2018-04-20 22:11:43,902 INFO sqlalchemy.engine.base.Engine select version()


2018-04-20 22:11:43,902 - sqlalchemy.engine.base.Engine - INFO - select version()


2018-04-20 22:11:43,905 INFO sqlalchemy.engine.base.Engine {}


...

2018-04-20 22:12:46,004 INFO sqlalchemy.engine.base.Engine {'uid': 40287333, 'first_name': None, 'last_name': None, 'is_sub': False, 'chose_hse': False, 'chose_mgimo': False, 'th_business': False, 'th_career': False, 'th_tech': False, 'th_arts': False, 'th_chill': False}


2018-04-20 22:12:46,004 - sqlalchemy.engine.base.Engine - INFO - {'uid': 40287333, 'first_name': None, 'last_name': None, 'is_sub': False, 'chose_hse': False, 'chose_mgimo': False, 'th_business': False, 'th_career': False, 'th_tech': False, 'th_arts': False, 'th_chill': False}


2018-04-20 22:12:46,007 INFO sqlalchemy.engine.base.Engine COMMIT


2018-04-20 22:12:46,007 - sqlalchemy.engine.base.Engine - INFO - COMMIT
2018-04-20 22:13:32,866 - telegram.ext.dispatcher - WARNING - A TelegramError was raised while processing the Update
2018-04-20 22:13:32,867 - telegram.ext.dispatcher - ERROR - An uncaught error was raised while handling the error
Traceback (most recent call last):
  File "/home/eamag/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/telegram/vendor/ptb_urllib3/urllib3/connectionpool.py", line 402, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/home/eamag/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/telegram/vendor/ptb_urllib3/urllib3/connectionpool.py", line 398, in _make_request
    httplib_response = conn.getresponse()
  File "/home/eamag/.pyenv/versions/3.6.0/lib/python3.6/http/client.py", line 1331, in getresponse
    response.begin()
  File "/home/eamag/.pyenv/versions/3.6.0/lib/python3.6/http/client.py", line 297, in begin
    ver

2018-04-20 22:16:11,058 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot355058372:AAHYrVaOhj9MROOL0J4bflm6HLUWHwPF6u0/getUpdates (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x7f9e917d5b38>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))
2018-04-20 22:16:11,060 - telegram.ext.dispatcher - ERROR - An uncaught error was raised while handling the error
Traceback (most recent call last):
  File "/home/eamag/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/telegram/ext/dispatcher.py", line 271, in process_update
    self.dispatch_error(None, update)
  File "/home/eamag/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/telegram/ext/dispatcher.py", line 384, in dispatch_error
    callback(self.bot, update, error)
TypeError: 'MessageHandler' object is not cal

http://docs.sqlalchemy.org/en/latest/orm/session_basics.html
todo - clean up